# Project3:  Wrangle OpenStreetMap Data
## Map area: Portland, Oregon

* Area map: https://www.openstreetmap.org/relation/186579#map=10/45.5424/-122.6541
* Data: OSM XML from https://mapzen.com/data/metro-extracts/metro/portland_oregon/ 

## Data wrangling
### Auditing street types
Overall, street names already look pretty clean. The original case study script cought some street types which were not included in the original list for expected street types, but I found many of them are valid street types. There were also many street names that end with a direction (e.g., Southwest Willamette Way East). I added more valid street types and "North","South","East", and "West" to the list of expected street types as followings.

In [ ]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", 
            "Square","Lane", "Road", "Trail", "Parkway", "Commons", "Path",
            "Run","Woods","Heights", "Broadway", "Circle", "Highway", 
            "Loop", "Terrace", "Way","Circus","Byway","Chantilly", 
            "Churchill","Alley","End","Curve", 
            "Crest","Point", "View","Summit","Downs","Preakness",
            "North","South","East","West"] 

There were still upexpected street types left to audit, but some of them were just unfamiliar street types which are likely to be valid. Other types left are as followings:
* Highway followed by a highway number: e.g., Southeast Highway 211. There are valid street names for an address. 
* Missing street types: e.g., 'Southwest 158th'. These are problematic street name since it is missing a street type. It was not feasible to update these one by one by internet map search. 
* Street names entered with errors: For example, there were full adresses or street names followed by an apartment number. I did not change these.
* Abbreviated street types: e.g., '4th Ave' or 'NE Glisan St.'. I updated these street names using the following mapping in my final cleaning step.

In [ ]:
mapping_street = { "St": "Street","St.": "Street","Rd": "Road",
                  "Rd.": "Road","Ave": "Avenue"}

### Auditing "k" values for node tags
I audited "k" values for node tags using the following function. 

In [ ]:
def audit_node_tag_k(filename):
    kvalues = set()
    for _, element in ET.iterparse(filename):
        if element.tag == 'node':
            for child in element.findall('tag'):  
                kvalues.add(child.attrib["k"])
    return kvalues

Some suspicious values I found are listed below:
* gnis:County_num vs. gnis:county_id 
* gnis:County vs. gnis:county_name
* 'Phone' vs. 'phone' (I will also check other values including 'phone')

Thus, I checked both "k" and "v" values of node tags with the suspicious "k" values using the following function.

In [ ]:
Suspicious = ["gnis:County_num","gnis:county_id", "gnis:County", 
              "gnis:county_name",'Phone','phone','contact:phone',
              'phone:alternate','phone:tollfree']

def audit_node_tag_k_suspicious(filename):
    kv_values = {}
    for _, element in ET.iterparse(filename):
        if element.tag == 'node':
            for child in element.findall('tag'):  
                if child.attrib["k"] in Suspicious: 
                    if child.attrib["k"] not in kv_values:
                        kv_values[child.attrib["k"]] = [child.attrib["v"]]
                    else:
                        if child.attrib["v"] not in  
                        kv_values[child.attrib["k"]]:
                            kv_values[child.attrib["k"]].append
                            (child.attrib["v"])
    return kv_values

Here is some part of the output:

In [ ]:
{'Phone': ['503-238-6330'],
 'contact:phone': ['+1-360-882-4082'],
 'gnis:County': ['Clackamas',...omitted...,'Yamhill'],
 'gnis:County_num': ['005', '009', '011', '051', '067', '047', '071'],
 'gnis:county_id': ['011', '059', '009', '067', '071', '051', '047'],
 'gnis:county_name': ['Marion',...omitted...,'Skamania'],
 'phone': ['+1-503-644-5748',
           '+1-503-595-1205',
           '+1-503-245-3183',
           '+1 (503) 282-9603',
           ...omitted...

* As I expected "gnis:County_num" and "gnis:county_id" seem to be used interchangeably, so do "gnis:County and gnis:county_name". The following website shows both "gnis:County_num" and "gnis:county_id" represent County FIPS code: 
http://wiki.openstreetmap.org/wiki/USGS_GNIS. The wiki page also says "County" in "gnis:County" is a county name.

* "Phone" and "phone" were used interchangeably. I will replace "Phone" with "phone".

* Most phone numbers are in this format '+1-###-###-####', but some numbers are not. For example, there are '503-238-6330','(360) 258-1713', '+01-503-639-1712' or '5032535327'. I will make every phone number to have this format '+1-###-###-####'.

Before I clean these, I audited way tags as well in the same way I did for node tags. 

### Auditing "k" values for way tags
I found several suspicious "k" values for way tags and checked their both "k" and "v" values as I did for node tags. Some of my suspicions were not confirmed. Here I only listed confirmed ones.

* 'servic' vs. 'service': It looks 'servic' was a typo. I will replace 'servic' with 'service'.
* 'dog' vs. 'dogs': They seem to be used interchangeably. I will replace 'dogs' with 'dog'.
* There are some phone numbers with formats different from '+1-###-###-####'. I will make every phone number to have this format '+1-###-###-####', but the special case like '503-683-5359 Fax: 503-683-4913' will not be changed.

### Updating problematic "k" values
I updated problematic "k" values using the following mapping. 

In [ ]:
mapping = {"gnis:County_num":"gnis:county_id", 
           "gnis:County":"gnis:county_name",
           "Phone":"phone","servic":"service", 
           "dogs": "dog"}

### Updating phone numbers
Phone numbers were changed into '+1-###-###-####' format if they were not in the format using the following regular expressions and function.

In [ ]:
good_phone_format = re.compile(r'^\+1\-(\d{3})-(\d{3})-(\d{4})$')
phone_format = re.compile(r'(\d{3})\D*(\d{3})\D*(\d{4})') 

def update_phone(phone_number, good_phone_format, phone_format):
    if not good_phone_format.search(phone_number):
        if 10 <= sum(char.isdigit() for char in phone_number) <= 12: 
            #only proper digits
            searched = phone_format.search(phone_number)
            if searched is not None: # If searched is None, it makes an error
                number_parsed = searched.groups()  
                phone_number = '+1-' + number_parsed[0]+'-'+number_parsed[1]
                +'-'+ number_parsed[2]           
    return phone_number

### Converting XML to CSV and creating database
The cleaned data was converted into 5 csv files. The Database OpenStreetMap_Portland.db was created and its 5 tables were created using the provided schema one by one using Python.

Note: The uploaded Python code is what I used up to here. From here I used my Command Prompt to run SQL.

## SQL Analysis
### Overview of the dataset

In [ ]:
sqlite> .dbinfo
database page size:  1024
write format:        1
read format:         1
reserved bytes:      0
file change counter: 15
database page count: 928804
freelist page count: 0
...omitted...

The size of the database is 928804 KB (929 MB). My OSM XML file was 1.54 GB.

In [ ]:
sqlite> SELECT Count(id) FROM nodes;
6577944
sqlite> SELECT Count(id) FROM ways;
850234

There are 6,577,944 nodes and 850,234 ways in total.

In [ ]:
sqlite> SELECT COUNT(DISTINCT(uid))
   ...> FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways);
1233

In [ ]:
sqlite> SELECT user, COUNT(*) as num
   ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways)
   ...> GROUP BY user
   ...> ORDER BY num DESC
   ...> LIMIT 5;
Peter Dobratz_pdxbuildings|1960041
lyzidiamond_imports|1904820
Mele Sax-Barnett|582424
baradam|540940
Darrell_pdxbuildings|433857

The number of unique users is 1,233 and the top 5 users are listed above.

### Addidtional  Statistics
#### Top  keys for nodes_tags and ways_tags

In [ ]:
sqlite> SELECT key, COUNT(*) as num
   ...> FROM nodes_tags
   ...> GROUP BY key
   ...> ORDER BY num DESC;

In [ ]:
highway|34959
street|30413
housenumber|30389
city|30133
postcode|30101
name|11801
amenity|10462
...omitted...

In [ ]:
sqlite> SELECT key, COUNT(*) as num
   ...> FROM ways_tags
   ...> GROUP BY key
   ...> ORDER BY num DESC
   ...> LIMIT 15;

In [ ]:
building|621020
ele|534058
street|432103
postcode|432023
housenumber|431908
city|431869
height|404184
levels|211923
highway|181228
name|112702
sidewalk|68170
zip_left|56927
zip_right|56279
county|49618
bicycle|40087

The top keys for nodes_tags and ways_tags separately were queried and some of them are listed above.

A lot of keys were inspected, but here I will only report some of the keys and their some values found to be most interesting in the following order:
* amenity 
* amenity: bicycle_parking 
* bicycle_parking 
* bicycle 
* amenity: recycling 

Note: Only the key 'bicycle' is from ways_tags. All others are from nodes tags. 

#### amenity

In [ ]:
sqlite> SELECT value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> WHERE key = 'amenity'
   ...> GROUP BY value
   ...> ORDER BY num DESC
   ...> LIMIT 10;

In [ ]:
bicycle_parking|2666
bench|868
waste_basket|805
place_of_worship|781
restaurant|666
fast_food|560
post_box|476
cafe|386
drinking_water|231
recycling|215

There were 98 kinds of amenities and the top 10 amenities are listed above. I further investigated some of the top amenities.

#### amenity:  bicycle_parking
The most frequent amenity value is 'bicycle_parking' and this is not surprising because Portland is one of the most bike-friendly cities in U.S. The following query shows 50 nodes were also tagged as a bicycle rental amenity.

In [ ]:
sqlite> SELECT COUNT(*)
   ...> FROM nodes_tags
   ...> WHERE key = 'amenity' and value = 'bicycle_rental';
50

#### bicycle_parking 

Actually, 'bicycle_parking' was also used as a key for node_tags and it was the 20th most frequent key. I found values for the key 'bicycle_parking' show types of bicycle parking from http://wiki.openstreetmap.org/wiki/Key:bicycle_parking. The below query shows that there are 7 types of bicycle parking, stands, bollard, rack, wave, wall_loops, building, and anchors in this map area. The majority of bicycle parkings have the stands type (94.8%). There was one incorrect value 'yes'.

In [ ]:
sqlite> SELECT value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> WHERE key = 'bicycle_parking'
   ...> GROUP BY value
   ...> ORDER BY num DESC;

In [ ]:
stands|2339
bollard|70
rack|27
wave|17
wall_loops|7
building|4
anchors|2
yes|1

#### bicycle (ways_tags)

In [ ]:
sqlite> SELECT value, COUNT(*) as num
   ...> FROM ways_tags
   ...> WHERE key = 'bicycle' and type = 'regular'
   ...> GROUP BY value
   ...> ORDER BY num DESC;

In [ ]:
designated|22789
yes|7003
no|3149
dismount|116
destination|15
permissive|13
preferred|7
avoid|6
shared_lane|5
private|4

The key 'bicycle' for ways_tags shows bicycle restrictions. Some of the values are explained on the website https://wiki.openstreetmap.org/wiki/Bicycle, but not all of them. The most frequent value is 'designated' (68.8%) which means  almost 70% of ways are specially designated for bicycle use. This seems to show Portland is a very bike-friendly city, but I cannot make a rigorous conclusion without comparing to other cities. 

#### amenity: recycling
I was glad to find 'recycling' is the 10th frequent amenity with 215 nodes_tags in Portland because Portland is known for its eco-friendliness. The original XML looks like the following where 'recycing' is the value for the key 'amenity'. 

In [ ]:
<tag k="amenity" v="recycling" />
<tag k="recycling:cans" v="yes" />
<tag k="recycling_type" v="container" />
<tag k="recycling:plastic_bottles" v="yes" />

In [ ]:
sqlite> SELECT nodes_tags.key, count(*) as num
   ...> FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags 
   ...> WHERE value = 'recycling') recycle
   ...> ON nodes_tags.id = recycle.id
   ...> WHERE nodes_tags.type = 'recycling' and nodes_tags.value = 'yes'
   ...> GROUP BY nodes_tags.key
   ...> ORDER BY num DESC;

In [ ]:
plastic_bottles|158
cans|145
paper|111
glass_bottles|88
clothes|25
glass|23
shoes|5
plastic|2
aluminium|1
books|1
cardboard|1
cooking_oil|1
eyeglasses|1
paper_bags|1
plastic_packaging|1
scrap_metal|1

The above query showed possible types of recycling each with the number of nodes. I did not know I can even recycle eyeglasses. With more queries, I found  eyeglasses can be recycled at Lions Clubs, an international organization. 

### Other ideas about the datasets

While I was trying many keys and their values, I found the following key:value's show where the datsets need major improvements. 
* amenity: cafe 
* shop: convience 

#### amenity:  cafe

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags 
   ...> WHERE value = 'cafe') cafe
   ...> ON nodes_tags.id = cafe.id
   ...> WHERE nodes_tags.key ='name'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC;

In [ ]:
Starbucks|129
Peet's|7
Black Rock|5
Jamba Juice|5
Peet's Coffee & Tea|5
Dutch Bros.|4
Peet's Coffee|3
Brewed Awakenings|2
Canteen|2
Dutch Bros. Coffee|2
Portland Roasting Coffee|2
Starbucks Coffee|2
Stumptown|2
...omitted...

I found 'cafe' is the 8th most frequent amenity value (386 nodes) in Portland. Starbucks (value = 'starbucks' or 'Starbucks Coffee') is the most frequent cafe (33.9% of cafe nodes). The second most frequent cafe was Peet's (value = 'Peet's', 'Peet's Coffee & Tea', or 'Peet's Coffee'). Here I noticed several cafe names are used to represent the same cafe. A similar problem was found in convenience store names. 

#### shop: convenience

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags JOIN (SELECT DISTINCT(id) FROM nodes_tags 
   ...> WHERE value = 'convenience') convenience
   ...> ON nodes_tags.id = convenience.id
   ...> WHERE nodes_tags.key ='name'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC
   ...> LIMIT 12;

In [ ]:
7-Eleven|24
Plaid Pantry|21
Food Mart|8
Your Northwest Travel Mart|4
711|3
The Market|2
50th Market|1
52nd Avenue Market|1
7 Eleven|1
7 Market|1
7-11|1
7/11|1

The key 'shop' was the 25th most frequent key for nodes_tags and its most frequent value is 'convenience', which represents convenience stores. The above query shows the top 12 convenience stores in Portland, but 5 of them, '7-Eleven', '711', '7 Eleven', '7-11', and '7/11', seem to represent the same store.

These chain store or company names like Starbucks or 7-Eleven seriously need to be cleaned up by standardizing their names. We cannot get query outputs we want using the current datasets. For example, the above output is actually showing 8 store names although 12 names were queried. One way to clean up these names is mapping. For example, mapping = {'711':'7-Eleven', '7 Eleven':'7-Eleven',  '7-11':'7-Eleven', '7/11':'7-Eleven'} can be used as I did for street names. Standardizing names will also make a thorough search for these names possible in this database. However, there are some anticipated problems. First of all, this will require extensive auditing since there are so many of those chain names in various places in the database. Moreover, there might be some similar but different names we do not know how to deal with. Standardizing these names for all areas in OpenStreetMap would be fantastic, but enormous efforts would be required (e.g., making a list of valid store names to guide users). 

## Conclusion

The OpenStreetMap data for Portland area was huge, but cleaner than I expected. There were only a few abbreviated street types to update and no problematic characters. Non-standardized phone nummbers and store names were major issues I found, but the phone numebers were easily fixed. The dataset gave a lot of useful information regarding recycling and bicycling. It also met my expectations that it will show environment and bicycle friendliness of Portland. 